In [32]:
import os
import snowflake.connector 
from snowflake.connector.pandas_tools import write_pandas
import pandas as pd 

In [33]:
sf_account_id = ''
sf_pw = os.environ['SF_PASSWORD']
sf_login = os.environ['SF_LOGIN']

DB_name = 'TEST_DB'
schema_name = 'PUBLIC' 
table_name = 'TEST_TABLE'

In [34]:
# Reading Data from CSV
df = pd.read_json('./anscombe.json')
df.columns = [x.upper() for x in df.columns]
df.head()

,SERIES,X,Y
0,I,10,8.04
1,I,8,6.95
2,I,13,7.58
3,I,9,8.81
4,I,11,8.33


In [35]:
# Creating Connection to Snowflake
cnx = snowflake.connector.connect(
    user=sf_login,
    password=sf_pw,
    account=sf_account_id,
    database=DB_name,
    schema=schema_name, 
    warehouse='COMPUTE_WH',
    role='ACCOUNTADMIN'
)

cs = cnx.cursor() 

In [43]:
# Create Table       
create_table = "create or replace TABLE TEST_DB.PUBLIC.TEST_TABLE (SERIES VARCHAR(16777216),X FLOAT,Y FLOAT);"
cs.execute(create_table)

In [44]:
# Write Table 
write_pandas(cnx, 
             df, 
             table_name=table_name)

/var/folders/md/b9gwzs7n1sn16v24b0j0fl780000gn/T/ipykernel_68333/3201502633.py:2: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  write_pandas(cnx,


(True,
 1,
 44,
 [('zjawiicjwo/file0.txt', 'LOADED', 44, 44, 1, 0, None, None, None, None)])

In [46]:
#SQL Query
cs.execute("SELECT * FROM TEST_DB.PUBLIC.TEST_TABLE LIMIT 10;")
rows = cs.fetchall()
for row in rows:
    print(row)

('I', 10.0, 8.04)
('I', 8.0, 6.95)
('I', 13.0, 7.58)
('I', 9.0, 8.81)
('I', 11.0, 8.33)
('I', 14.0, 9.96)
('I', 6.0, 7.24)
('I', 4.0, 4.26)
('I', 12.0, 10.84)
('I', 7.0, 4.8100000000000005)


In [48]:
# Dataframe from Snowflake
sql = "SELECT * FROM TEST_DB.PUBLIC.TEST_TABLE;"
cs.execute(sql)

df_from_df = cs.fetch_pandas_all()
df_from_df

,SERIES,X,Y
0,I,10.0,8.04
1,I,8.0,6.95
2,I,13.0,7.58
3,I,9.0,8.81
4,I,11.0,8.33
5,I,14.0,9.96
6,I,6.0,7.24
7,I,4.0,4.26
8,I,12.0,10.84
9,I,7.0,4.81


In [49]:
cs.close()
cnx.close()